In [2]:
import pandas as pd
import requests
import time

In [7]:
df = pd.read_csv('punkty_poi.csv', sep=";")
print(df)

                                                  Nazwa            Typ  \
0     Szkoła Podstawowa nr 33 im. Stefana Banacha w ...         school   
1       Akademickie Liceum Ogólnokształcące "Lingwista"         school   
2      XXIV Liceum Ogólnokształcące im. Stefana Banacha         school   
3              Szkoła Podstawowa nr 5 im. Jana Pawła II         school   
4         XIV Liceum Ogólnokształcące im. Jana Pawła II         school   
...                                                 ...            ...   
1981                                  Barzychowskiej 02  stop_position   
1982                                  Barzychowskiej 01  stop_position   
1983                                      Knyszyńska 02  stop_position   
1984                               Obrońców Wybrzeża 02  stop_position   
1985                                    Kokoszki PKP 02  stop_position   

       Latitude  Longitude  
0     54.420949  18.479074  
1     54.360706  18.644899  
2     54.420860  18.4791

In [8]:

# Funkcja odwrotnego geokodowania z Nominatim (OpenStreetMap)
def reverse_geocode_osm(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lat,
        "lon": lon,
        "format": "json",
        "addressdetails": 1
    }
    headers = {
        "User-Agent": "OpenAI-Geocoder-Script"
    }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        if response.status_code == 200:
            address = response.json().get("address", {})
            city = address.get("city") or address.get("town") or address.get("village") or ""
            district = address.get("suburb") or address.get("city_district") or ""
            return pd.Series([city, district])
    except Exception as e:
        print(f"Błąd przy {lat}, {lon}: {e}")
    return pd.Series(["", ""])


In [9]:
# Tworzenie pustych kolumn
df[["miasto", "dzielnica"]] = ""

In [ ]:
# Iteracja po wierszach z limitem czasowym (1 zapytanie / sekundę zgodnie z regulaminem Nominatim) (zajmuje to około 42 minuty)
for idx, row in df.iterrows():
    miasto, dzielnica = reverse_geocode_osm(row["Latitude"], row["Longitude"])
    df.at[idx, "miasto"] = miasto
    df.at[idx, "dzielnica"] = dzielnica
    print(f"{idx + 1}/{len(df)}: {miasto} - {dzielnica}")
    time.sleep(1)  # ograniczenie API

1/1986: Gdańsk - Osowa
2/1986: Gdańsk - Śródmieście
3/1986: Gdańsk - Osowa
4/1986: Gdańsk - Zaspa-Rozstaje
5/1986: Gdańsk - Zaspa-Rozstaje
6/1986: Gdańsk - Wyspa Sobieszewska
7/1986: Gdańsk - Osowa
8/1986: Gdańsk - Wrzeszcz Górny
9/1986: Gdańsk - Chełm
10/1986: Gdynia - Chwarzno-Wiczlino
11/1986: Gdańsk - Ujeścisko - Łostowice
12/1986: Gdynia - Karwiny
13/1986: Gdańsk - Piecki-Migowo
14/1986: Gdańsk - Piecki-Migowo
15/1986: Gdańsk - Śródmieście
16/1986: Gdańsk - Śródmieście
17/1986: Gdynia - Orłowo
18/1986: Gdańsk - Suchanino
19/1986: Gdańsk - Chełm
20/1986: Gdańsk - Wrzeszcz Górny
21/1986: Gdańsk - Wrzeszcz Górny
22/1986: Gdańsk - VII Dwór
23/1986: Gdańsk - Zaspa-Młyniec
24/1986: Gdańsk - Młyniska
25/1986: Gdańsk - Siedlce
26/1986: Gdańsk - Siedlce
27/1986: Sopot - 
28/1986: Gdańsk - Brzeźno
29/1986: Gdańsk - Brzeźno
30/1986: Gdańsk - Żabianka-Wejhera-Jelitkowo-Tysiąclecia
31/1986: Gdańsk - Żabianka-Wejhera-Jelitkowo-Tysiąclecia
32/1986: Gdańsk - Chełm
33/1986: Gdańsk - Wrzeszcz Dolny

In [11]:
# Zapisz do nowego pliku
df.to_csv("punkty_poi_z_miastem.csv", sep=";", index=False)
print("Gotowe! Zapisano do 'punkty_poi_z_miastem.csv'")

Gotowe! Zapisano do 'punkty_poi_z_miastem.csv'
